In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install GetoldTweets3

In [0]:
import GetOldTweets3 as got
import pandas as pd
import os

###Created Function for Extracting tweets from Tweeter using GetoldTweets3 package

###Extract tweets for every 10 days 
###There is maximum capacity of 1000 tweets that Getoldtweets return in every Request. So for every keyword extracting 1000 max tweets between given dates


In [0]:
#We need to set SetSince and SetUntil dates for extracting tweets between particular dates 

def text_query_to_csv(text_query, count):
    # Creation of query object
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query)\
                                               .setMaxTweets(count).setSince('2020-03-21').setUntil('2020-03-24').setLang("en")#.setNear('Chicago,United States')
    
    #tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query).setSince(since_date).setUntil(until_date).setMaxTweets(count)
    # Creation of list that contains all tweets
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    # Creating list of chosen tweet data
    text_tweets = [[tweet.date,tweet.username,tweet.geo, tweet.text] for tweet in tweets]

    # Creation of dataframe from tweets
    tweets_df = pd.DataFrame(text_tweets, columns = ['Datetime', 'Text','Text','Text',])
    # Converting tweets dataframe to csv file
    #tweets_df.to_csv('{}-{}k-tweets.csv'.format(text_query, int(count/1000)), sep=',')
    os.chdir('/content/drive/My Drive/IDS 594/Code/final code')
    tweets_df.to_csv("GetOldTweets24March.csv",mode='a',header=False)
    #tweets_df.to_csv("NovGetoldtweets.csv",mode='a',header=False)
    #tweets_df


In [0]:
#Extracting tweets which has following Cannabis related keywords
search_words=['cannabisedibles',
'Marijuana',
'cannabis',
'weed',
'joint',
'blunt ',
'mary jane',
'hash',
'bong',
'cbd',
'ganja',
'hemp',
'kush',
'marihuana'
]


In [0]:
for i in search_words:
    text_query = i
    count = 1000
    text_query_to_csv(text_query, count)

###Reading all 378000 tweets captured using above function to preprocess them and further filter them based on tweets mentioning medical claims about using Cannabis

In [0]:
import pandas as pd
import os

file = '/content/drive/My Drive/IDS 594/Code/final code/GetOldTweets24March.csv'#All 380000 tweets
data=pd.read_csv(file)
print(data.count())
data.head()



Id            378347
Date          378347
Username      378347
Unnamed: 3         0
Tweet         378347
dtype: int64


,Id,Date,Username,Unnamed: 3,Tweet
0,0,2020-01-04 21:34:03+00:00,TranquiloEssen5,NaN,TRANQUILO HEMP OIL The best quality at the bes...
1,1,2020-01-04 02:49:22+00:00,justhighlo,NaN,Just in time for your weekend brunch. Making b...
2,2,2020-01-03 15:02:01+00:00,WSPS_NEWS,NaN,Cannabis edibles are appearing on the shelves!...
3,3,2020-01-03 01:48:52+00:00,sdcannabis,NaN,[New Product] emeraldskyedibles Come by and ge...
4,4,2020-01-02 20:07:03+00:00,cannabis_mag,NaN,Cannabis Edibles Continue to Boom https://buff...


###Cleaning the tweet and extracting keywords(tokens) from tweets

In [0]:
import re
import nltk
def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•' 
def clean_tweet(tweet):
    #tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet_token_list = [word for word in tweet.split(' ')]
    return tweet_token_list

data['Tweet_tokens'] = data.Tweet.apply(clean_tweet)
data.head(20) 

,Id,Date,Username,Unnamed: 3,Tweet,Tweet_tokens
0,0,2020-01-04 21:34:03+00:00,TranquiloEssen5,NaN,TRANQUILO HEMP OIL The best quality at the bes...,"[tranquilo, hemp, oil, the, best, quality, at,..."
1,1,2020-01-04 02:49:22+00:00,justhighlo,NaN,Just in time for your weekend brunch. Making b...,"[just, in, time, for, your, weekend, brunch, m..."
2,2,2020-01-03 15:02:01+00:00,WSPS_NEWS,NaN,Cannabis edibles are appearing on the shelves!...,"[cannabis, edibles, are, appearing, on, the, s..."
3,3,2020-01-03 01:48:52+00:00,sdcannabis,NaN,[New Product] emeraldskyedibles Come by and ge...,"[new, product, emeraldskyedibles, come, by, an..."
4,4,2020-01-02 20:07:03+00:00,cannabis_mag,NaN,Cannabis Edibles Continue to Boom https://buff...,"[cannabis, edibles, continue, to, boom, the, a..."
5,5,2020-01-02 18:56:26+00:00,CaritoTranquilo,NaN,TRANQUILO HEMP OIL The best quality at the bes...,"[tranquilo, hemp, oil, the, best, quality, at,..."
6,6,2020-01-01 14:58:58+00:00,SixthWaveNano,NaN,Government of Canada: Final regulations: Edibl...,"[government, of, canada, final, regulations, e..."
7,0,2020-01-04 23:59:58+00:00,lilvvvvv,NaN,its an old dr office someone bought n turned i...,"[ts, an, old, dr, office, someone, bought, n, ..."
8,1,2020-01-04 23:59:42+00:00,Weed4highC,NaN,"Buy Marijuana online USA,mail order marijuana ...","[buy, marijuana, online, usa, mail, order, mar..."
9,2,2020-01-04 23:59:23+00:00,CannabisBizNews,NaN,Lucas plans to introduce bills to ‘relax’ the ...,"[lucas, plans, to, introduce, bills, to, ‘rela..."


###Joining keywords (tokens) from tweet to get cleaned tweet  

In [0]:
def join_tweetTokens(tweet_tokens):
  cleaned_tweet=' '.join(tweet_tokens)
  return cleaned_tweet

data['cleaned_tweet'] = data.Tweet_tokens.apply(join_tweetTokens)
data

,Id,Date,Username,Unnamed: 3,Tweet,Tweet_tokens,cleaned_tweet
0,0,2020-01-04 21:34:03+00:00,TranquiloEssen5,NaN,TRANQUILO HEMP OIL The best quality at the bes...,"[tranquilo, hemp, oil, the, best, quality, at,...",tranquilo hemp oil the best quality at the bes...
1,1,2020-01-04 02:49:22+00:00,justhighlo,NaN,Just in time for your weekend brunch. Making b...,"[just, in, time, for, your, weekend, brunch, m...",just in time for your weekend brunch making ba...
2,2,2020-01-03 15:02:01+00:00,WSPS_NEWS,NaN,Cannabis edibles are appearing on the shelves!...,"[cannabis, edibles, are, appearing, on, the, s...",cannabis edibles are appearing on the shelves ...
3,3,2020-01-03 01:48:52+00:00,sdcannabis,NaN,[New Product] emeraldskyedibles Come by and ge...,"[new, product, emeraldskyedibles, come, by, an...",new product emeraldskyedibles come by and get ...
4,4,2020-01-02 20:07:03+00:00,cannabis_mag,NaN,Cannabis Edibles Continue to Boom https://buff...,"[cannabis, edibles, continue, to, boom, the, a...",cannabis edibles continue to boom the article ...
...,...,...,...,...,...,...,...
378342,995,2019-12-30 20:10:08+00:00,thetotalreport,NaN,How To Use Marijuana In Public Housing Without...,"[how, to, use, marijuana, in, public, housing,...",how to use marijuana in public housing without...
378343,996,2019-12-30 20:10:03+00:00,LADailyJournal,NaN,The legal landscape of banking #marijuana busi...,"[the, legal, landscape, of, banking, #marijuan...",the legal landscape of banking #marijuana busi...
378344,997,2019-12-30 20:10:02+00:00,cheddar,NaN,Fox News host @TuckerCarlson had some choice w...,"[fox, news, host, @tuckercarlson, had, some, c...",fox news host @tuckercarlson had some choice w...
378345,998,2019-12-30 20:09:52+00:00,isiah4congress,NaN,REAL criminal justice reform looks like: Demil...,"[real, criminal, justice, reform, looks, like,...",real criminal justice reform looks like demili...


In [0]:
#Bag of words with all cannabis related terms
Cannabis_Keywords=['cannabisedibles',
'marijuana',
'cannabis',
'cannabis legislation',
'weed',
'pot ',
'joint',
'blunt ',
'mary jane',
'hash',
'bong',
'budder',
'cbd',
'ganja',
'hemp',
'indica',
'kush',
'marihuana ',
'reefer',
'sativa',
'thc',
'edibles',
'bud',
'dab',
'dabber',
'weededibles',
'recweed',
'medweed',
'medicalmarijuana']

###Building a 3 phase classifier to check if tweet is making Cannabis related Heathclaim or not. 
####In the first step checking if keyword from 1st bag (Cannabis related words) exist in the tweet 

In [0]:
#If keyword from Cannabis bag of words exist or not
data['Cannabis_or_Not']=data.apply(lambda row: 'yes' if any(word in row['cleaned_tweet'] for word in Cannabis_Keywords) else 'no' ,axis=1)

data_Cannabis=data.loc[data['Cannabis_or_Not'] == 'yes']
data_Cannabis


,Id,Date,Username,Unnamed: 3,Tweet,Tweet_tokens,cleaned_tweet,Cannabis_or_Not
0,0,2020-01-04 21:34:03+00:00,TranquiloEssen5,NaN,TRANQUILO HEMP OIL The best quality at the bes...,"[tranquilo, hemp, oil, the, best, quality, at,...",tranquilo hemp oil the best quality at the bes...,yes
1,1,2020-01-04 02:49:22+00:00,justhighlo,NaN,Just in time for your weekend brunch. Making b...,"[just, in, time, for, your, weekend, brunch, m...",just in time for your weekend brunch making ba...,yes
2,2,2020-01-03 15:02:01+00:00,WSPS_NEWS,NaN,Cannabis edibles are appearing on the shelves!...,"[cannabis, edibles, are, appearing, on, the, s...",cannabis edibles are appearing on the shelves ...,yes
3,3,2020-01-03 01:48:52+00:00,sdcannabis,NaN,[New Product] emeraldskyedibles Come by and ge...,"[new, product, emeraldskyedibles, come, by, an...",new product emeraldskyedibles come by and get ...,yes
4,4,2020-01-02 20:07:03+00:00,cannabis_mag,NaN,Cannabis Edibles Continue to Boom https://buff...,"[cannabis, edibles, continue, to, boom, the, a...",cannabis edibles continue to boom the article ...,yes
...,...,...,...,...,...,...,...,...
378342,995,2019-12-30 20:10:08+00:00,thetotalreport,NaN,How To Use Marijuana In Public Housing Without...,"[how, to, use, marijuana, in, public, housing,...",how to use marijuana in public housing without...,yes
378343,996,2019-12-30 20:10:03+00:00,LADailyJournal,NaN,The legal landscape of banking #marijuana busi...,"[the, legal, landscape, of, banking, #marijuan...",the legal landscape of banking #marijuana busi...,yes
378344,997,2019-12-30 20:10:02+00:00,cheddar,NaN,Fox News host @TuckerCarlson had some choice w...,"[fox, news, host, @tuckercarlson, had, some, c...",fox news host @tuckercarlson had some choice w...,yes
378345,998,2019-12-30 20:09:52+00:00,isiah4congress,NaN,REAL criminal justice reform looks like: Demil...,"[real, criminal, justice, reform, looks, like,...",real criminal justice reform looks like demili...,yes


####Phase 2- Checking if Tweet is medical related or not by checking If keyword from Medical Condition bag of words exist or not

In [0]:
#list of medical conditions
file1 = '/content/drive/My Drive/IDS 594/Code/final code/Medical Condition Bag.csv'
Medical_condition=pd.read_csv(file1)
print("Total =",Medical_condition.count())
Medical_condition=Medical_condition['Medical condition'].str.lower()
#print("\n",Medical_condition.head(5))
print(type(Medical_condition))

Medical_Condition_list = Medical_condition.tolist()
Medical_Condition_list

Total = Medical condition    963
dtype: int64
<class 'pandas.core.series.Series'>


['seizures',
 'nausea',
 'vomiting',
 'multiple sclerosis',
 'cancer chemotherapy',
 'glaucoma',
 'headache',
 'cancer',
 'lennox-gastaut syndrome',
 'dravet syndrome',
 'hiv',
 'aids',
 'nerve injury',
 'nerve damage',
 'opioid use',
 'anxiety',
 'pressure in  eye',
 'inflammatory bowel disease',
 'ulcerative colitis',
 'crohn disease',
 'irritable bowel syndrome',
 'ibs',
 'abdominal pain',
 'bowel movements',
 'movement disorder',
 'tourette syndrome',
 'posttraumatic stress disorder',
 'ptsd',
 'sleep problems',
 'amyotrophic lateral sclerosis',
 'als',
 'alzheimerâ€™s disease',
 'behavioral disturbances',
 'dementia',
 'autism',
 'autism behavioural',
 'tics',
 'depression',
 'restlessness',
 'rage attacks',
 'arthritis',
 'ra',
 'inflammation in joints',
 'cachexia anorexia',
 'cystic fibrosis',
 'fibromyalgia',
 'hepatitis c',
 'migraine',
 'parkinsonâ€™s disease',
 'peripheral neuropathy',
 'psoriasis',
 'cancer growth',
 'refractory seizures',
 'diarrhea',
 'coronovirus',
 'fo

In [0]:

data_Cannabis['Medical_Or_Not']=data_Cannabis.apply(lambda row: 'yes' if any(word in row['cleaned_tweet'] for word in Medical_Condition_list) else 'no' ,axis=1)

data_Medical_Condition=data_Cannabis.loc[data_Cannabis['Medical_Or_Not'] == 'yes']
#data_Medical_Condition
#data_Medical_Condition.to_csv("Removed_Medical_Condition_Cannabis.csv",header=True,mode='a', encoding='utf-8')

####Phase 3-Checking if Tweet is mentioning effect of using Cannabis on Medical condition by checking if tweet has word from "Effect" bag of words

In [0]:
#Tweet has effect of medical condition or not
file1 = '/content/drive/My Drive/IDS 594/Code/final code/Effects Bag.csv'
Medical_condition_effects=pd.read_csv(file1)
print("Total effects =",Medical_condition_effects.count())
Medical_condition_effects=Medical_condition_effects['effects'].str.lower()
#print("\n",Medical_condition_effects.head(5))
print(type(Medical_condition_effects))

Medical_condition_effects_list = Medical_condition_effects.tolist()
Medical_condition_effects_list

Total effects = effects    30
dtype: int64
<class 'pandas.core.series.Series'>


['tremor',
 'treat',
 'loss of appetite',
 'weight loss',
 'heal',
 'relief',
 'neuropathic',
 'help',
 'decrease',
 'reduce',
 'lower',
 'symptom',
 'significant',
 'severity',
 'effective',
 'improve',
 'ease',
 'curb',
 'lower',
 'stimulat',
 'reduce',
 'grow',
 'slow',
 'therapy',
 'cure',
 'kill',
 'ehance',
 'manage',
 'well',
 'good']

In [0]:
#If keyword from Effect of medical condition bag of words exist or not
#file3 = '/content/drive/My Drive/IDS 594/Code/final code/Final Consolidated Medical Cannabis Tweets.csv'
#data_Medical_Condition=pd.read_csv(file3)
data_Medical_Condition['Effect_exist_or_Not']=data_Medical_Condition.apply(lambda row: 'yes' if any(word in row['cleaned_tweet'] for word in Medical_condition_effects_list) else 'no' ,axis=1)

data_Cannabis_Medical_Condition_effects=data_Medical_Condition.loc[data_Medical_Condition['Effect_exist_or_Not'] == 'yes']
data_Cannabis_Medical_Condition_effects


,Rownumber,Id,Date,Username,Unnamed: 3,Tweet,Tweet_tokens,Medical_Or_Not,cleaned_tweet,prob bot,Bot_or_Not,Effect_exist_or_Not
1,35,28,2020-01-04 23:50:58+00:00,Dsk19691,NaN,"I once was adamant against legalizing, until m...","['i', 'once', 'was', 'adamant', 'against', 'le...",yes,i once was adamant against legalizing until my...,0.007751,Not a Bot,yes
2,94,87,2020-01-04 23:31:49+00:00,BraddockPhil007,NaN,iâ€™d rather use marijuana/ thc/ and cbd than ...,"['â€™d', 'rather', 'use', 'marijuana', 'thc', ...",yes,â€™d rather use marijuana thc and cbd than use...,0.003838,Not a Bot,yes
4,212,205,2020-01-04 23:00:15+00:00,MarijuanaBreak,NaN,The therapeutic benefits of Snow Lotus include...,"['the', 'therapeutic', 'benefits', 'of', 'snow...",yes,the therapeutic benefits of snow lotus include...,0.637516,Bot,yes
7,504,497,2020-01-04 21:48:41+00:00,Darq_Marc,NaN,I support the Marijuana Justice Act of 2017 be...,"['i', 'support', 'the', 'marijuana', 'justice'...",yes,i support the marijuana justice act of because...,0.010862,Not a Bot,yes
10,701,694,2020-01-04 20:58:52+00:00,centerinventor1,NaN,Like they use the climate children for their o...,"['like', 'they', 'use', 'the', 'climate', 'chi...",yes,like they use the climate children for their o...,0.084960,Not a Bot,yes
...,...,...,...,...,...,...,...,...,...,...,...,...
7631,378090,743,2019-12-30 21:04:55+00:00,MimiTexasAngel,NaN,#PresidentSanders #MedicareForAll #Cannabis #G...,"['#presidentsanders', '#medicareforall', '#can...",yes,#presidentsanders #medicareforall #cannabis #g...,0.013465,Not a Bot,yes
7632,378097,750,2019-12-30 21:03:14+00:00,AsianMJane,NaN,I treat my anxiety with medical marijuana,"['i', 'treat', 'my', 'anxiety', 'with', 'medic...",yes,i treat my anxiety with medical marijuana,0.001218,Not a Bot,yes
7633,378149,802,2019-12-30 20:56:05+00:00,chris_danyow,NaN,I agree Cigarettes are cancer sticks and they ...,"['i', 'agree', 'cigarettes', 'are', 'cancer', ...",yes,i agree cigarettes are cancer sticks and they ...,0.006744,Not a Bot,yes
7635,378164,817,2019-12-30 20:54:06+00:00,MimiTexasAngel,NaN,"45,000+ deaths a year because no #Healthcare 5...","['', 'deaths', 'a', 'year', 'because', 'no', '...",yes,deaths a year because no #healthcare bankrupt...,0.013465,Not a Bot,yes


###Now we have filtered tweets to only those which mention particular Medical claim about using the Cannabis by Using 3 phase clssifier

In [0]:
#Exporting the output 
import os
os.chdir('/content/drive/My Drive/IDS 594/Code/final code')
data_Cannabis_Medical_Condition_effects.to_csv("Final_Cannabis_Effects_MedicalConditions_BOW_MedicalCannabisTweets.csv",header=True,mode='a', encoding='utf-8')

### Final code for checking if tweet is fact or misinformation

In [0]:
#Import final tweets
import pandas as pd
import os

#file = '/content/drive/My Drive/IDS 594/Code/final code/Final_Cannabis_Effects_MedicalConditions_BOW_MedicalCannabisTweets.csv'#All 380000 tweets
#data=pd.read_csv(file)
data=data_Cannabis_Medical_Condition_effects
print("Total rows in data=",len(data))
data


Total rows in data= 2587


,Rownumber,Id,Date,Username,Unnamed: 3,Tweet,Tweet_tokens,cleaned_tweet,probability of bot,Bot_or_Not
0,35,28,2020-01-04 23:50:58+00:00,Dsk19691,NaN,"I once was adamant against legalizing, until m...","['i', 'once', 'was', 'adamant', 'against', 'le...",i once was adamant against legalizing until my...,0.007751,Not a Bot
1,94,87,2020-01-04 23:31:49+00:00,BraddockPhil007,NaN,iâ€™d rather use marijuana/ thc/ and cbd than ...,"['â€™d', 'rather', 'use', 'marijuana', 'thc', ...",â€™d rather use marijuana thc and cbd than use...,0.003838,Not a Bot
2,212,205,2020-01-04 23:00:15+00:00,MarijuanaBreak,NaN,The therapeutic benefits of Snow Lotus include...,"['the', 'therapeutic', 'benefits', 'of', 'snow...",the therapeutic benefits of snow lotus include...,0.637516,Bot
3,504,497,2020-01-04 21:48:41+00:00,Darq_Marc,NaN,I support the Marijuana Justice Act of 2017 be...,"['i', 'support', 'the', 'marijuana', 'justice'...",i support the marijuana justice act of because...,0.010862,Not a Bot
4,858,851,2020-01-04 20:16:38+00:00,lab_ou,NaN,Together elevated ptsd symptoms and marijuana ...,"['together', 'elevated', 'ptsd', 'symptoms', '...",together elevated ptsd symptoms and marijuana ...,0.034857,Not a Bot
...,...,...,...,...,...,...,...,...,...,...
2582,376182,835,2019-12-30 18:20:45+00:00,hollysundstrom2,NaN,#NAME?,"['', '@selenagomez', 'high', 'cbd', 'low', 'th...",@selenagomez high cbd low thc hemp is now leg...,0.048246,Not a Bot
2583,376191,844,2019-12-30 18:18:01+00:00,hollysundstrom2,NaN,#NAME?,"['', '@selenagomez', 'share', 'on', 'pinterest...",@selenagomez share on pinterest hemp oil cont...,0.048246,Not a Bot
2584,376194,847,2019-12-30 18:17:18+00:00,hollysundstrom2,NaN,@selenagomez I just ordered some from USA onli...,"['@selenagomez', 'i', 'just', 'ordered', 'some...",@selenagomez i just ordered some from usa onli...,0.048246,Not a Bot
2585,376207,860,2019-12-30 18:10:06+00:00,HempJersey,NaN,"Meet Glyn, our soil expert. Using his methods,...","['meet', 'glyn', 'our', 'soil', 'expert', 'usi...",meet glyn our soil expert using his methods we...,0.377760,Not a Bot


In [0]:
#Importing Facts sheet which has Verified medical claims about
Facts=pd.read_csv('/content/drive/My Drive/IDS 594/Code/final code/Copy of Facts classified new.csv')
Facts

,Index,MedicalUsage,Cannabis,Medical Condition,Effect,direction
0,1,Epidiolex has CBD treatment of seizures assoc...,NaN,Lennox-Gastaut syndrome,seizures,"help,relieve,support,benefit,comfort,improve,s..."
1,2,Marinol and Syndros THC drugs used are used t...,NaN,cancer,nausea,"help,relieve,support,benefit,comfort,improve,s..."
2,3,Marinol and Syndros THC drugs used are used t...,NaN,cancer,vomiting,"help,relieve,support,benefit,comfort,improve,s..."
3,4,used to treat loss of appetite and weight los...,NaN,HIV,appetite,"grow,enlarge,expand,swell,rise,escalate,soar,s..."
4,5,used to treat loss of appetite and weight los...,NaN,HIV,weight loss,"help,relieve,support,stop,benefit,comfort,impr..."
5,6,used to treat loss of appetite and weight los...,NaN,AIDS,appetite,"grow,enlarge,expand,swell,rise,escalate,soar,s..."
6,7,used to treat loss of appetite and weight los...,NaN,AIDS,weight loss,"help,relieve,support,stop,benefit,comfort,impr..."
7,8,particularly neuropathic pain associated with...,NaN,neuropathic pain,NaN,"help,relieve,support,benefit,stop,comfort,impr..."
8,9,"Cannabinoids, primarily CBD, have been studied...",NaN,epilepsy,seizures,"help,relieve,support,benefit,comfort,improve,s..."
9,10,cannabis or substances derived from it could l...,NaN,Galucoma,lower pressure,"help,relieve,support,benefit,comfort,improve,s..."


###Checking if the tweet is claiming Medical use of Cannabis mentioned in the Fact sheet to classify tweet as Fact or Misinformqtion

In [0]:
#Classifier for classifying tweets as Facts or Misinformation using direction(positive or negative)
Medical_condition=Facts['Medical Condition'].tolist()
effect=Facts['Effect'].tolist()

Medical_condition=[x.lower().strip() for x in Medical_condition]    
#print(Medical_condition)

import numpy as np
effect=[' ' if x is np.nan else x.lower() for x in effect]
effect

list1=[]
for i in range(len(Facts['direction'])):
    list2=[]
    list_new=Facts['direction'][i].split(",")
    for j in range(len(list_new)):
        list2.append(list_new[j])
    list1.append(list2)
    
def Fact_or_not(tweet):
    for i in range(len(Medical_condition)):
        if Medical_condition[i] in tweet:
            if effect[i] in tweet:
                if any(word in tweet for word in list1[i]):
                    return 'Fact'
                    
    return 'Misinformation'

data['Fact_or_Misinformation']=data['cleaned_tweet'].apply(Fact_or_not)

import os
os.chdir('/content/drive/My Drive/IDS 594/Code/final code')
data.to_csv("Tweets classified Facts or MisInformation.csv",header=True,mode='a', encoding='utf-8')

#29 tweets out of 2588 are classified as Facts and remaining are classified as misinformation